```{currentmodule} optimap
```

In [ ]:
from optimap.utils import jupyter_render_animation as render

```{tip}
Download this tutorial as a {download}`Jupyter notebook <converted/activation.ipynb>`, or a {download}`python script <converted/activation.py>` with code cells. We highly recommend using [Visual Studio Code](#vscode) to execute this tutorial.
```

# Tutorial 6: Activation Maps

This tutorial demonstrates how to compute local activation times and activation maps from cardiac optical mapping data using ``optimap``. Local activation times (often referred to as LATs) are times at which the tissue becomes electrically activated. 

Computing local activation times corresponds to determining when the optical signal in a given pixel passes a certain pre-defined threshold or intensity value. For instance, if the optical trace is normalized and fluctuates betwen [0,1] then the tissue could be defined as being 'electrically activated' when the time-series rises above or below 0.5 (depending on the fluorescent indicator and polarity of the signal).

Here, we will use an example data from {cite:t}`Rybashlykov2022` in which a planar action potential wave propagates across the ventricle of a mouse heart.

In [ ]:
import optimap as om
import monochrome as mc
import numpy as np
import matplotlib.pyplot as plt

filename = om.download_example_data("doi:10.5281/zenodo.5557829/mouse_41_120ms_control_iDS.mat")
video = om.load_video(filename)
metadata = om.load_metadata(filename)
print(f"Loaded video with shape {video.shape} and metadata {metadata}")
frequency = metadata["frequency"]

The `mouse_41_120ms_control_iDS.mat` file from the [Zenodo dataset](https://doi.org/10.5281/zenodo.5557829) shows a induced pacing beats in a mouse heart. The {func}`load_metadata` function loads the metadata from the MATLAB file, in this case the acquisition frame rate. We visualize the video using [Monochrome](https://github.com/sitic/Monochrome):

In [ ]:
# Show video
mc.show(video, name=filename.name, metadata=metadata)

In [ ]:
from IPython.display import Video
mp4_file = om.download_example_data("mouse_41_120ms_control_MiCAM_monochrome.mp4", silent=True)
Video(filename=mp4_file, embed=True, html_attributes="controls autoplay loop")

We use ``optimap``'s {func}`background_mask` function to blanck out the background in the image, such that the activation map is only computed for pixels showing tissue. 

In [ ]:
# remove background by masking
mask = om.background_mask(video[0], show=False)
mask = om.image.dilate_mask(mask, iterations=5, show=False)
om.image.show_mask(mask, video[0], title="Background mask")

In [ ]:
new_mask = om.interactive_mask(image=video[0], initial_mask=mask)
om.save_mask('mouse_41_120ms_control_MiCAM_monochrome_mask.png', new_mask)
mask = new_mask
mask = om.load_mask('mouse_41_120ms_control_MiCAM_monochrome_mask.png')

In [ ]:
video_filtered = om.video.smooth_spatiotemporal(video, sigma_temporal=1, sigma_spatial=1)
video_filtered = om.video.mean_filter(video_filtered, size_spatial=5)

# Normalize the video using a pixelwise sliding window
video_norm = om.video.normalize_pixelwise_slidingwindow(video_filtered, window_size=200)
video_norm[:, mask] = np.nan

Because the mouse heart was stained with the voltage-sensitive dye Di-4-ANEPPS, the tissue becomes darker when it depolarizes (negative signal / polarity):

In [ ]:
om.show_video_pair(video, video_norm, title1="original video",
                   title2="normalized video", interval=100)

# Or in Monochrome:
#
# mc.show(video, name="original video")
# mc.show(video_norm, name="normalized video")

In [ ]:
render(lambda: om.show_video_pair(video, video_norm, title1="original video", title2="normalized video", interval=250))

In [ ]:
crossings = om.activation.find_activations(1 - video_norm, fps=frequency)
print(f"Found {len(crossings)} activation events at frames: {crossings}")

Let's plot some of the video frames as the wave propagates across the ventricles:

In [ ]:
figure, axs = plt.subplots(1, 6, figsize=(10, 3))
axs[0].imshow(video[0], cmap='gray')
axs[0].set_title('original')
axs[0].set_axis_off()

for i in range(1, 6):
    t = i * 2
    axs[i].imshow(video_norm[crossings[0] - 7 + t], cmap='gray', vmin=0, vmax=1)
    axs[i].set_axis_off()
    time = t * (1000/frequency)  # convert to ms
    axs[i].set_title(f"{time:.1f} ms")
plt.axis('off')
plt.show()

We can now compute an activation map by identifying the local activation times in each pixel that correspond to when the action potential wave front passes through that pixel.

## Computing Activation Maps from Pixel-wise Normalized Optical Maps

We will first compute an activation map with a pixel-wise normalized video. The pixel-wise normalized video contains values between 0 and 1:

In [ ]:
om.print_properties(video_norm)

A pixel-wise normalization was sufficient as opposed to a sliding-window pixel-wise normalization, see [Tutorial 2](signal_extraction.ipynb), because we isolated a short part of the video that is only 20 frames long. In other cases it might be necessary to use a sliding-window pixel-wise normalization or a frame-wise difference video (e.g. with motion), see below.

Let's plot some of the optical traces (manually selected so that they show locations which become subsequently activated):

In [ ]:
# positions = om.select_positions(video[0])
positions =  [(134, 101), (15, 93), (94, 99), (53, 97)]
fig, axs = plt.subplots(1, 2, figsize=(10,5))
om.trace.show_positions(positions, video[0], ax=axs[0])
traces = om.extract_traces(video_norm,
                           positions,
                           size=10,
                           fps=frequency,
                           ax=axs[1])
axs[1].axhline(y=0.5, color='r', linestyle='dashed', label='threshold')
axs[1].text(0.03, 0.52, 'threshold', color='r')
plt.xlim(0, 0.12)
plt.show()

We can use ``optimap``'s {func}`compute_activation_map` function to automatically compute a two-dimensional activation map which shows the local activation times in every pixel:

In [ ]:
idx = crossings[7]
activation_map = om.compute_activation_map(video_norm[idx - 7:idx + 10], inverted=True, fps=frequency, vmax=13)
levels = [3, 6, 9, 12, 15]
video_norm2 = video_norm.copy()
video_norm2[np.isnan(video_norm2)] = np.nanmean(video_norm2)
video_norm2 = om.video.mean_filter(video_norm2, size_spatial=5)
activation_map = om.compute_activation_map(video_norm[idx - 9:idx + 10], inverted=True, fps=frequency, vmax=18, show_contours=True, contour_levels=levels)
activation_map = om.compute_activation_map(video_norm2[idx - 9:idx + 10], inverted=True, fps=frequency, vmax=18, show_contours=True, contour_levels=levels)

Note that we used the argument `inverted=True` due to the negative polarity of the signal ($- \Delta F / F$). If me had manually inverted the video beforehand or with calcium imaging data this would not be necessary. The range of local activation times can be displayed with:

In [ ]:
om.print_properties(activation_map)

In this case, the local activation times are given in milliseconds (based on argument `fps=500`) and they range between 0ms and 36ms. The function {func}`compute_activation_map` uses {func}`show_image` to plot the activation map (which can be disabled with argument `show=False`):

In [ ]:
om.show_activation_map(activation_map, cmap="jet", title='Activation Map', show_colorbar=True, colorbar_title='Activation Time [ms]', vmax=18)

We have plotted the activation map using the `jet` colormap, here are some other options:

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(8, 3))
om.show_activation_map(activation_map, cmap='jet', show_colorbar=True, title='cmap=jet', ax=axs[0], colorbar_title=None)
om.show_activation_map(activation_map, cmap='magma', show_colorbar=True, title='cmap=magma', ax=axs[1], colorbar_title=None)
om.show_activation_map(activation_map, cmap='twilight_shifted', show_colorbar=True, title='cmap=twilight_shifted', ax=axs[2], colorbar_title=None)
plt.suptitle('Activation maps with different colormaps')
plt.show()

## Computing Activation Maps from Frame-Wise Difference Optical Maps

In [Tutorial 2](signal_extraction.ipynb), we introduced the frame-wise difference method to emphasize sudden temporal changes in a video. Sudden temporal changes are caused by upstrokes of the action potential or calcium transients and the frame-wise difference filter is therefore ideally suited to visualize wavefronts as they propagate across the tissue.

In [ ]:
video_diff = om.video.temporal_difference(video_warped, 5)
video_diff[:, mask] = np.nan
video_diff_norm = om.video.normalize_pixelwise(video_diff)

The frame-wise difference approach enhances action potential upstroke, see the following video with temporal difference in the middle and our previous pixel-wise normalized video on the right:

In [ ]:
om.show_videos([video, video_diff_norm, video_warped_norm],
               titles=["original", "warped, frame-wise diff", "warped, normalized"],
               interval=100)

In [ ]:
render(lambda: om.show_videos([video, video_diff_norm, video_warped_norm],
               titles=["original", "warped, frame-wise diff", "warped, normalized"],
               interval=250))

Let's visualize the wavefront as an overlay over the raw (motion-stabilized) video. We will need to further post-process the data as follows: 

In [ ]:
video_diff[video_diff > 0] = 0
video_diff_norm = om.video.normalize_pixelwise(-video_diff)

The action potential upstroke overlaid onto the raw video:

In [ ]:
om.video.show_video_overlay(video_warped,
                            overlay=video_diff_norm,
                            vmin_overlay=-1,
                            vmax_overlay=1)

In [ ]:
render(lambda: om.video.show_video_overlay(video_warped, video_diff_norm, vmin_overlay=-1, vmax_overlay=1, interval=200))

## Adding Contour Lines to Activation Maps

Contour lines are a powerful visualization tool that can help highlight the wavefront propagation. They connect points with the same activation time, making it easier to visualize the speed and direction of propagation. Let's add contour lines to our activation map:

In [ ]:
# Create a figure with the activation map and contour lines
fig, ax = plt.subplots(figsize=(8, 6))

# Show the activation map as a colored background
im = ax.imshow(activation_map, cmap='jet')

# Add contour lines with labels
# The levels parameter controls at which activation times contour lines are drawn
contour = ax.contour(activation_map, levels=np.arange(0, 35, 5), colors='white', linewidths=1)

# Add labels to the contour lines (time in ms)
plt.clabel(contour, inline=True, fontsize=8, fmt='%1.0f ms')

# Add colorbar and title
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('Activation Time [ms]')
ax.set_title('Activation Map with Contour Lines')
ax.axis('off')
plt.tight_layout()
plt.show()

You can also combine contour lines with a raw image to visualize the propagation path over the heart tissue:

In [ ]:
# Create a figure showing both the raw image and the contour lines
fig, ax = plt.subplots(figsize=(8, 6))

# Show the raw image as background
ax.imshow(video_warped[0], cmap='gray')

# Add contour lines on top
# Use more levels for finer detail
contour_levels = np.arange(0, 35, 3)
contour = ax.contour(activation_map, levels=contour_levels, 
                    cmap='jet', linewidths=1.5, alpha=0.8)

# Add a colorbar
cbar = plt.colorbar(contour, ax=ax)
cbar.set_label('Activation Time [ms]')

ax.set_title('Activation Contour Lines Overlaid on Tissue')
ax.axis('off')
plt.tight_layout()
plt.show()

Customizing Contour Visualization

You can customize various aspects of the contour lines to highlight different features:

In [ ]:
# Create a more customized visualization
fig, ax = plt.subplots(figsize=(8, 6))

# Apply a different colormap to the background
im = ax.imshow(activation_map, cmap='viridis', alpha=0.7)

# Create contour lines with custom levels and style
contour_levels = np.arange(0, 35, 2.5)  # More contour lines for finer detail
contour = ax.contour(activation_map, levels=contour_levels, 
                     colors='white', linewidths=0.8)

# Add contour labels selectively (only to some contour lines)
plt.clabel(contour, inline=True, fontsize=8, 
           fmt='%1.0f ms', levels=np.arange(0, 35, 5))

# Add masked tissue outline
# First, create a mask boundary line
mask_boundary = ax.contour(mask, levels=[0.5], colors='red', linewidths=1.5)

# Add colorbar and annotations
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('Activation Time [ms]')
ax.set_title('Detailed Activation Map with Contour Lines')
ax.axis('off')
plt.tight_layout()
plt.show()

Extracting Isochronal Lines for Analysis

You can also extract the contour lines as paths for further analysis:

In [ ]:
# Create contour lines for analysis
contour_set = plt.contour(activation_map, levels=np.arange(0, 35, 5))

# No need to show this plot - just getting the contours
plt.close()

# Extract paths from a specific contour level (e.g., 15 ms isochronal line)
contour_level_index = 3  # This is the index for the 15 ms contour in our sequence
contour_paths = contour_set.collections[contour_level_index].get_paths()

print(f"Number of paths in the 15 ms contour: {len(contour_paths)}")

# Example: Plot just the 15 ms isochronal line on the raw image
fig, ax = plt.subplots(figsize=(8, 6))
ax.imshow(video_warped[0], cmap='gray')

for path in contour_paths:
    # Extract vertices of the path
    vertices = path.vertices
    ax.plot(vertices[:, 0], vertices[:, 1], 'r-', linewidth=2, label='15 ms isochrone')
    
# Remove duplicate labels
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())

ax.set_title('15 ms Isochronal Line')
ax.axis('off')
plt.tight_layout()
plt.show()

These contour lines are especially useful for estimating conduction velocity and identifying areas of slow conduction or conduction block in the heart tissue. </file>

This enhancement adds a comprehensive section about contour lines to the activation maps tutorial. The additions include:

Basic contour line overlay on the activation map
Overlaying contour lines on the original tissue image
Creating more customized contour visualizations with different styling options
Extracting isochronal lines (contour paths) for further analysis
These examples demonstrate various ways to visualize wave propagation with contour lines, which is an essential technique when analyzing cardiac activation patterns. The contour lines make it easier to see how the activation wave spreads across the tissue and can help identify conduction disorders or other cardiac abnormalities.

We will now compute an activation map from the frame-wise difference video:

```{warning}
This tutorial is currently work in progress. We will add more information soon.
```